# Sensitivity across cancer cell lines
We evaluated catalogued relationships in MOAlmanac across cancer cell lines. This notebook should be run from the MOAlmanac directory. 

In [1]:
import pandas as pd
import tinydb

almanac = tinydb.TinyDB('/Users/brendan/Github/moalmanac/moalmanac/datasources/moalmanac/moalmanac.json')
almanac_genes = almanac.table('genes').all()[0]['genes']

In [ ]:
paper_root = ''
variant = pd.read_csv('datasources/preclinical/ccle.variants.evaluated.txt', sep='\t')
cna = pd.read_csv('datasources/preclinical/ccle.copy-numbers.evaluated.txt', sep='\t')
fusion = pd.read_csv('datasources/preclinical/sanger.fusions.evaluated.txt', sep='\t')

variant = variant.loc[:, ['feature', 'alteration_type', 'alteration']].drop_duplicates()
variant['feature_type'] = 'Somatic Variant'
variant = variant[variant['feature'].isin(almanac_genes)].reset_index(drop=True)

cna = cna.loc[:, ['feature', 'alteration_type', 'alteration']].drop_duplicates()
cna['feature_type'] = 'Copy Number'
cna = cna[cna['feature'].isin(almanac_genes)].reset_index(drop=True)

fusion = fusion.loc[:, ['feature', 'alteration_type', 'alteration']].drop_duplicates()
fusion['feature_type'] = 'Rearrangement'
fusion = fusion[fusion['feature'].isin(almanac_genes)].reset_index(drop=True)

In [ ]:
%load_ext autoreload
%autoreload 2

from config import COLNAMES
from config import CONFIG

from annotator import Annotator, Almanac
from datasources import Datasources
from features import Features

def annotate_somatic(df, dbs, ontology='Unknown'):
    df[Annotator.score_bin] = Annotator.preallocate_bin(Annotator.score_bin, df.index)
    df = Almanac.annotate(df, dbs, ontology)
    return df

dbs = Datasources.generate_db_dict(CONFIG)
    
variant = Features.preallocate_missing_columns(variant)
annotated_variant = annotate_somatic(variant, dbs)

In [ ]:
cna = Features.preallocate_missing_columns(cna)
annotated_cna = annotate_somatic(cna, dbs)

In [ ]:
fusion = Features.preallocate_missing_columns(fusion)
annotated_fusion = annotate_somatic(fusion, dbs)

In [ ]:
annotated_somatic = pd.concat([annotated_variant, annotated_cna, annotated_fusion], ignore_index=True)
annotated_somatic.to_csv('annotated_somatic.ccle.txt', sep='\t')

In [ ]:
%load_ext autoreload
%autoreload 2

import evaluator

In [ ]:
def evaluate_somatic(df):
    df = evaluator.Evaluator.evaluate_almanac(df)
    return df

evaluated_somatic = evaluate_somatic(annotated_somatic)

In [ ]:
evaluated_germline = Features.create_empty_dataframe()
evaluated_ms_variants = Features.create_empty_dataframe()
evaluated_ms_status = Features.create_empty_dataframe()
evaluated_burden = Features.create_empty_dataframe()
evaluated_burden.loc[0, 'a'] = ''
evaluated_signatures = Features.create_empty_dataframe()
evaluated_wgd = Features.create_empty_dataframe()

actionable = evaluator.Actionable.evaluate(evaluated_somatic, 
                                           evaluated_germline,
                                           evaluated_ms_variants,
                                           evaluated_ms_status,
                                           evaluated_burden,
                                           evaluated_signatures,
                                           evaluated_wgd
                                          )

In [ ]:
%load_ext autoreload
%autoreload 2

import annotator
import datasources
import investigator

In [ ]:
actionable['alteration'].fillna('', inplace=True)

In [ ]:
idx = actionable[actionable['feature_type'].eq('Rearrangement') & (actionable['alteration'].eq('') | actionable['alteration'].eq('None--None'))].index
actionable.loc[idx, 'alteration'] = actionable.loc[idx, 'feature'] + '--None'

In [ ]:
actionable[actionable['feature_type'].eq('Rearrangement')]

In [ ]:
dbs_preclinical = datasources.Preclinical.import_dbs()
efficacy_dictionary = investigator.SensitivityDictionary.create(dbs_preclinical, actionable, 'cell lines')
efficacy_summary = investigator.SummaryDataFrame.create(efficacy_dictionary, actionable, 'cell lines')
#annotated_actionable = annotator.PreclinicalEfficacy.annotate(actionable, efficacy_summary)

In [ ]:
efficacy_summary_all = efficacy_summary.copy(deep=True)
efficacy_summary = efficacy_summary[~efficacy_summary['tested_subfeature'].str.contains("None")]

In [ ]:
efficacy_summary.to_csv('cell-lines.efficacy-summary.txt', sep='\t', index=False)

In [ ]:
annotated_actionable = annotator.PreclinicalEfficacy.annotate(actionable, efficacy_summary)
annotated_actionable.to_csv('cell-lines.annotated.actionable.txt', sep='\t', index=False)

In [ ]:
efficacy_summary['pvalue_mww'].astype(float).le(0.05).value_counts()

In [ ]:
efficacy_summary['pvalue_mww_float'] = efficacy_summary['pvalue_mww'].astype(float)
tmp = (efficacy_summary
 .loc[:, ['feature_display', 'therapy_name', 'pvalue_mww_float']]
 .sort_values('pvalue_mww_float')
 .drop_duplicates(['feature_display', 'therapy_name'])
)

tmp[tmp['pvalue_mww_float'].lt(0.05)]#.index

In [ ]:
tmp_sig = (efficacy_summary
 .loc[tmp[tmp['pvalue_mww_float'].lt(0.05)].index, :]
 .loc[:, ['tested_subfeature', 'therapy_name', 'pvalue_mww_float']]
 .drop_duplicates(['tested_subfeature', 'therapy_name'])
)

In [ ]:
tmp_sig['therapy_name'].drop_duplicates().shape

In [ ]:
tmp_sig[tmp_sig['tested_subfeature'].str.contains('p.D594G')]

In [ ]:
efficacy_summary[efficacy_summary['feature_display'].str.contains('D594G')]

In [ ]:
efficacy_summary.loc[5710, :]

In [ ]:
import openpyxl
!mkdir -p ~/Github/moalmanac-paper/tables/supplementary-table-04
efficacy_summary.to_excel('~/Github/moalmanac-paper/tables/supplementary-table-04/supplementary-table-4.xlsx', index=False)

## Calculations

In [1]:
import pandas as pd
summary = pd.read_csv('tables/sensitivity/cell-lines.efficacy-summary.txt', sep='\t', low_memory=False)

In [6]:
summary[summary['pvalue_mww'].le(0.05)]['therapy_name'].drop_duplicates().sort_values().shape

(18,)

In [3]:
summary[summary['pvalue_mww'].le(0.05)]['therapy_name'].drop_duplicates().sort_values().tolist()

['AZD8186',
 'Afatinib',
 'Alectinib',
 'Alpelisib',
 'Bortezomib',
 'Bosutinib',
 'Buparlisib',
 'Dabrafenib',
 'Gefitinib',
 'KRAS (G12C) Inhibitor-12',
 'Olaparib',
 'Osimertinib',
 'Palbociclib',
 'Pictilisib',
 'Selumetinib',
 'Sorafenib',
 'Sunitinib',
 'Trametinib']